In [86]:
import pandas as pd

x = pd.read_pickle('../ml/signal_stat/buy_stat_1h.pkl')
y = pd.read_pickle('../ml/signal_stat/sell_stat_1h.pkl')
x = pd.concat([x, y])

In [93]:
import pandas as pd
from datetime import timedelta
from tqdm.auto import tqdm

def get_file(ticker):
    ''' Find files buy ticker names, file names can be in different formats '''
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    try:
        tmp_df_1h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_1h.pkl')
        tmp_df_4h = pd.read_pickle(f'../optimizer/ticker_dataframes/{ticker[:-4]}-{ticker[-4:]}-SWAP_4h.pkl')
    except FileNotFoundError:
        pass
    else:
        return tmp_df_1h, tmp_df_4h
    
    return None, None


def create_train_df(df, type, target_offset, first, last, step):
    ''' Create train dataset from signal statistics and ticker candle data'''
    train_df = pd.DataFrame()
    tickers = df['ticker'].unique()
    
    for ticker in tqdm(tickers):
        signal_df = df[df['ticker'] == ticker]
        times = signal_df['time']
        
        tmp_df_1h, tmp_df_4h = get_file(ticker)

        for t in times.to_list():
            pass_cycle = False
            row = tmp_df_1h.loc[tmp_df_1h['time'] == t, :].reset_index(drop=True)
            
            for i in range(first, last + step, step):
                time_prev = t + timedelta(hours= -i)
                try:
                    row_tmp = tmp_df_1h.loc[tmp_df_1h['time'] == time_prev, :].reset_index(drop=True)
                    row_tmp.columns = [c + f'_prev_{i}' for c in row_tmp.columns]
                except IndexError:
                    pass_cycle = True
                    break
                row = pd.concat([row, row_tmp.iloc[:,1:]], axis=1)
                
            if pass_cycle:
                continue
            
            time_next = t + timedelta(hours=target_offset)
            if type == 'buy':
                target = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'high'].reset_index(drop=True)
            else:
                target = tmp_df_1h.loc[tmp_df_1h['time'] == time_next, 'low'].reset_index(drop=True)

            target.name = 'target'
            rows = pd.concat([row, target], axis=1)
            
            if train_df.shape[0] == 0:
                train_df = rows
            else:
                train_df = pd.concat([train_df, rows])
    
    return train_df

# for how long time (in hours) we want to predict
target_offset = 24
# first previous data point to collect for model training (value represents number of hours before signal point)
first = 1
# last previous data point to collect for model training (value represents number of hours before signal point)
last = 10
# step of previous data points collecting (total number of points to collect is (last - first + step) / step)
step = 1
# dataset with the signal statistics
df = pd.read_pickle('signal_stat/buy_stat_1h.pkl')
tmp = pd.read_pickle('../ml/signal_stat/sell_stat_1h.pkl')
df = pd.concat([df, tmp])
# dataset for model train
train_df = create_train_df(df, 'buy', target_offset, first, last, step)
train_df = train_df.dropna().sort_values('time').reset_index(drop=True)
train_df.head()


100%|██████████| 374/374 [01:00<00:00,  6.18it/s]


,time,open,high,low,close,volume,open_prev_1,high_prev_1,low_prev_1,close_prev_1,...,high_prev_9,low_prev_9,close_prev_9,volume_prev_9,open_prev_10,high_prev_10,low_prev_10,close_prev_10,volume_prev_10,target
0,2022-12-21 23:00:00,0.04220,0.042300,0.04200,0.042200,916271.000,0.042600,0.042600,0.042200,0.042300,...,0.043000,0.042800,0.04290,751674.000,0.042700,0.042900,0.042700,0.042900,604272.000,0.043200
1,2022-12-22 03:00:00,246.20000,248.100000,245.80000,248.000000,8174.000,246.600000,247.700000,245.200000,246.200000,...,249.200000,247.800000,249.10000,11987.502,249.000000,249.900000,247.600000,248.300000,12723.229,246.200000
2,2022-12-23 07:00:00,0.69300,0.777000,0.69100,0.768000,516397.700,0.693000,0.696000,0.688000,0.695000,...,0.656000,0.653000,0.65600,20277.600,0.663000,0.667000,0.653000,0.653000,94242.300,0.730000
3,2022-12-23 07:00:00,1.07400,1.100000,1.07300,1.085000,310312.721,1.082000,1.084000,1.074000,1.074000,...,1.038000,1.027000,1.03500,24770.684,1.022000,1.029000,1.018000,1.029000,26867.020,1.086000
4,2022-12-23 19:00:00,0.05791,0.058277,0.05768,0.058167,395965.000,0.058276,0.058441,0.057649,0.057916,...,0.059364,0.059013,0.05923,566941.000,0.059387,0.059476,0.059013,0.059064,846843.000,0.057341


In [95]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()

train_cols = [c for c in train_df.columns if c not in ['time', 'target']]

X, y = train_df[train_cols], train_df['target']
X = scaler.fit_transform(X)
# y =  scaler.fit_transform(y.values.reshape(-1, 1))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression(positive=True)
lr.fit(X_train, y_train)

preds = lr.predict(X_val)
# preds = scaler.inverse_transform(preds)
# y_val = scaler.inverse_transform(y_val)

mean_squared_error(y_val, preds, squared=False)

24.694775778796973

In [85]:
train_df[train_cols].loc[:,lr.coef_ > 0]

,high,close,volume_prev_1
0,1.100000,1.085000,76595.593
1,0.777000,0.768000,35608.300
2,0.916400,0.911600,146720.670
3,1.292000,1.290000,43202.800
4,146.100000,146.000000,1443.995
...,...,...,...
3397,0.004374,0.004337,3448154.000
3398,0.446000,0.444000,7844.400
3399,4.520000,4.520000,14085.430
3400,0.446000,0.444000,7844.400


In [67]:
import lightgbm as lgb

scaler = StandardScaler()

X, y = train_df[train_cols], train_df['target']
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

lgbr = lgb.LGBMRegressor()
lgbr.fit(X_train, y_train)

preds = lgbr.predict(X_val)
mean_squared_error(y_val, preds, squared=False)

/home/alex/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.18503467367774978

In [110]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()

X, y = train_df[train_cols], (train_df['target'] > train_df['open']).map({True: 1, False: 0})
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

logr = LogisticRegression(C=0.1, max_iter=100000, class_weight='balanced')
logr.fit(X_train, y_train)

preds = logr.predict(X_val)

accuracy_score(y_val, preds)

0.44990892531876137

In [97]:
X, y = train_df[train_cols], (train_df['target'] > train_df['open']).map({True: 1, False: 0})

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

lgbc = lgb.LGBMClassifier()
lgbc.fit(X_train, y_train)

preds = lgbc.predict(X_val)
accuracy_score(y_val, preds)

0.6530054644808743